In [123]:
import os
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession

In [124]:
#set the PySpark environment veriable
os.environ['JAVA_HOME']="E:\jdk-22"
os.environ['SPARK_HOME'] = "E:\spark-3.5.1-bin-hadoop3"
os.environ['PYSPARK_DRIVER_PYTHON'] = "jupyter"
os.environ['PYSPARK_DRIVER_PYTHON_OPS'] = "notebook"
os.environ['PYSPARK_PYTHON'] = "python"

In [125]:
#Sart the Sparksession
spark = SparkSession.builder.appName("Practice")\
.config("spark.jars","E:\jdbc-driver\postgresql-42.7.3.jar")\
.config("spark.jars.packages","com.mysql:mysql-connector-j:8.4.0")\
.enableHiveSupport()\
.getOrCreate()

In [126]:
sc=spark.sparkContext
sc

<SparkContext master=local[*] appName=Practice>

In [127]:
# Query table using jdbc()
df = spark.read\
    .jdbc("jdbc:mysql://localhost:3306/park", "item",\
          properties={"user": "root", "password": "root@123", "driver":"com.mysql.cj.jdbc.Driver"})

In [129]:
df.show()

+---+------------+--------+----+---+-----+
| id|product_name|category|size|qty|price|
+---+------------+--------+----+---+-----+
|  1|        pent| trauser|  32| 10|  250|
|  2|        pent| trauser|  38| 10|  250|
|  3|        pent| trauser|  30| 10|  250|
|  4|        pent| trauser|  32| 10|  250|
|  5|     T-short| T-short|  32| 10|  250|
|  6|  half-short| T-short|  30| 10|  250|
|  7|      snkier|    show|  32| 10|  250|
|  8|      snkier| trauser|  30| 10|  250|
|  9|      snkier| trauser|  30| 10|  250|
| 10|      snkier| trauser|  35| 10|  250|
| 11|        pent| trauser|  32| 10|  250|
| 12|        pent| trauser|  32| 10|  250|
| 13|        pent| trauser|  32| 10|  250|
+---+------------+--------+----+---+-----+



In [130]:
jdbc_url = 'jdbc:postgresql://localhost:5432/mldb'
props = {'user': 'postgres',
              'password': 'postgres',
               'driver': 'org.postgresql.Driver'
             }
sql_df = spark.read.jdbc(url=jdbc_url, table='employee', properties=props)

In [131]:
sql_df.show()

+---+-----------------+---------------+---+--------------------+--------+
| id|             name|        company|age|             address|  salary|
+---+-----------------+---------------+---+--------------------+--------+
|101|    Krishna Rajan|      Accenture| 35|#Banglaore sasadr...|120000.0|
|102|     Surendra Rai|            IBM| 40|#Banglaore, singa...|180000.0|
|103|       Goopy Nath|Mphasis Limited| 35|#Banglaore, RT Na...| 90000.0|
|104|     Manish Thapa|Mantree Limited| 35|#Banglaore Bamana...|102000.0|
|105|Bina Kumar jhakri|            IBM| 35|#Banglaore HSR La...|150000.0|
|106|   Roshan Chetrry|       Indexcel| 35|#Banglaore, Korma...|110000.0|
+---+-----------------+---------------+---+--------------------+--------+



In [132]:
import psycopg2
# PostgreSQL connection properties
jdbc_url = "jdbc:postgresql://localhost:5432/mldb"
connection_properties = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}
# Fetch all table names from the PostgreSQL database
def get_table_names():
    conn = psycopg2.connect(
        dbname="mldb",
        user="postgres",
        password="postgres",
        host="localhost",
        port="5432"
    )
    cursor = conn.cursor()
    cursor.execute(
        "SELECT table_name FROM information_schema.tables WHERE table_schema='public';"
    )
    tables = cursor.fetchall()
    cursor.close()
    conn.close()
    return [table[0] for table in tables]

# Read all tables into PySpark DataFrames
table_names = get_table_names()

In [133]:
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|         |  df_item|       true|
+---------+---------+-----------+



In [134]:
df_all_table = pd.DataFrame({'Tables': table_names})

In [135]:
spark.createDataFrame(df_all_table).show()

+------------+
|      Tables|
+------------+
|    employee|
|       users|
|user_message|
|        post|
+------------+



In [136]:
user_df = spark.read.jdbc(url=jdbc_url, table='users', properties=connection_properties)
user_df.show()

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|surendra         ...|
|  2|Bikash thapa     ...|
|  3|Ritu Rai         ...|
|  4|Minakshi         ...|
+---+--------------------+



In [137]:
user_message_df = spark.read.jdbc(url=jdbc_url, table='post', properties=connection_properties)
user_message_df.show()

+---+--------------------+---------------+--------------------+--------------------+--------------------+
| id|                uuid|       username|             message|          created_at|          updated_at|
+---+--------------------+---------------+--------------------+--------------------+--------------------+
|  1|1719f9a6-a6f8-460...|   prity thapat|He is a full stac...|2023-02-11 21:01:...|2023-02-11 21:01:...|
|  2|590e1b1c-cae8-48a...|  krisha thapat|He is a full stac...|2023-02-11 21:02:...|2023-02-11 21:02:...|
|  3|3ffcd235-81e7-4bc...|   Binah thapat|He is a full stac...|2023-02-11 21:02:...|2023-02-11 21:02:...|
|  4|4067c132-3e25-492...|     sita thapa|He is a full stac...|2023-02-11 21:02:...|2023-02-11 21:02:...|
|  6|03cad32e-1e40-4be...|  binisha thapa|He is a full stac...|2023-02-11 21:02:...|2023-02-11 21:02:...|
|  8|8077dafc-1e8b-477...|   Bikash thapa|He is a full stac...|2023-02-11 21:03:...|2023-02-11 21:03:...|
| 10|9ccbe731-18cb-495...| diskarta thapa|He i

In [143]:
employee_df = spark.read.jdbc(url=jdbc_url, table='employee', properties=connection_properties)
employee_df.show()

+---+-----------------+---------------+---+--------------------+--------+
| id|             name|        company|age|             address|  salary|
+---+-----------------+---------------+---+--------------------+--------+
|101|    Krishna Rajan|      Accenture| 35|#Banglaore sasadr...|120000.0|
|102|     Surendra Rai|            IBM| 40|#Banglaore, singa...|180000.0|
|103|       Goopy Nath|Mphasis Limited| 35|#Banglaore, RT Na...| 90000.0|
|104|     Manish Thapa|Mantree Limited| 35|#Banglaore Bamana...|102000.0|
|105|Bina Kumar jhakri|            IBM| 35|#Banglaore HSR La...|150000.0|
|106|   Roshan Chetrry|       Indexcel| 35|#Banglaore, Korma...|110000.0|
+---+-----------------+---------------+---+--------------------+--------+



In [139]:
# employee_df = spark.read.jdbc(url=jdbc_url, table='user_message', properties=connection_properties)
# employee_df.show()

In [140]:
# Using Spark SQL Data Analysis the project
df.createOrReplaceTempView("df_item")

In [141]:
spark.sql("SELECT * FROM df_item where size > 30").show()

+---+------------+--------+----+---+-----+
| id|product_name|category|size|qty|price|
+---+------------+--------+----+---+-----+
|  1|        pent| trauser|  32| 10|  250|
|  2|        pent| trauser|  38| 10|  250|
|  4|        pent| trauser|  32| 10|  250|
|  5|     T-short| T-short|  32| 10|  250|
|  7|      snkier|    show|  32| 10|  250|
| 10|      snkier| trauser|  35| 10|  250|
| 11|        pent| trauser|  32| 10|  250|
| 12|        pent| trauser|  32| 10|  250|
| 13|        pent| trauser|  32| 10|  250|
+---+------------+--------+----+---+-----+

